In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
import torch.nn as nn

from modules.unet import UNet
from modules.percep_nets import ResidualsNetUp, ResidualsNetDown
from transfers import Transfer, pretrained_transfers
from logger import Logger, VisdomLogger
from datasets import load_train_val, load_test, load_ood, load_all, TaskDataset
from task_configs import tasks, RealityTask
from model_configs import model_types
from models import DataParallelModel
from utils import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_CUDA = torch.cuda.is_available()

In [4]:
logger = VisdomLogger("Jupyter_vis", env="YKIVVA_jupyter_visualize_", port=PORT, server=SERVER)

Setting up a new session...


No deletion
In (git) scaling-reset
Logging to environment YKIVVA_jupyter_visualize_


# DOWNLOAD MODELS

In [39]:
project_dir = "/scratch/kivva/projects/consistency_LS"
model_class = "resnet_based"
model_type = model_types[model_class]

pipeline = "multitask"
res_dir = os.path.join(project_dir, model_class, pipeline,)
dir_multitask = os.path.join(res_dir, "results_YKIVVA_resnet3_multitask:x->n&r_v1_/models")

pipeline = "direct"
res_dir = os.path.join(project_dir, model_class, pipeline,)
dir_direct = os.path.join(res_dir, "results_YKIVVA_resnet3_direct:n->r&r->n_v1_/models")

pipeline = "consistency"
res_dir = os.path.join(project_dir, model_class, pipeline,)
dir_consistency = os.path.join(res_dir, "results_YKIVVA_resnet3_perceptual:x->n&r_direct_v1_/models")


In [40]:
rgb_down = DataParallelModel(model_type["rgb"]["down"][0]())
normal_down = DataParallelModel(model_type["normal"]["down"][0]())
depth_down = DataParallelModel(model_type["depth_zbuffer"]["down"][0]())
normal_up = DataParallelModel(model_type["normal"]["up"][0]())
depth_up = DataParallelModel(model_type["depth_zbuffer"]["up"][0]())

In [41]:
path = dir_multitask+"/rgb2LS.pth"
if os.path.isfile(path):
    rgb_down.load_weights(path)
    print(1)

path = dir_direct + "/normal2LS.pth"
if os.path.isfile(path):
    normal_down.load_weights(path)
    print(2)

path = dir_direct+"/depth_zbuffer2LS.pth"
if os.path.isfile(path):
    depth_down.load_weights(path)
    print(3)

path = dir_multitask+"/LS2normal.pth"
if os.path.isfile(path):
    normal_up.load_weights(path)
    print(4)

path = dir_multitask+"/LS2depth_zbuffer.pth"
if os.path.isfile(path):
    depth_up.load_weights(path)
    print(5)

1
2
3
4
5


In [42]:
rgb2depth, path = pretrained_transfers[('rgb', 'depth_zbuffer')]
rgb2depth = DataParallelModel.load(rgb2depth(), path)

rgb2normal, path = pretrained_transfers[('rgb', 'normal')]
rgb2normal = DataParallelModel.load(rgb2normal(), path)

normal2depth, path = pretrained_transfers[('normal', 'depth_zbuffer')]
normal2depth = DataParallelModel.load(normal2depth(), path)

depth2normal, path = pretrained_transfers[('depth_zbuffer', 'normal')]
depth2normal = DataParallelModel.load(depth2normal(), path)

# ACCURACY

## DIRECT

In [43]:
src_task = tasks.rgb
target_task = tasks.depth_zbuffer
tasks_set = [src_task, target_task]

buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

test_iter = iter(test_loader)

accuracy = 0
for in_data, out_data in test_iter:
    in_data = in_data.to(DEVICE)
    out_data = out_data.to(DEVICE)
    if src_task==tasks.rgb:
        model_down = rgb_down.to(DEVICE)
        if target_task==tasks.normal:
            model = rgb2normal.to(DEVICE)
        if target_task==tasks.depth_zbuffer:
            model = rgb2depth.to(DEVICE)
    if src_task==tasks.normal:
        model_down = normal_down.to(DEVICE)
        model = normal2depth.to(DEVICE)
    if src_task==tasks.depth_zbuffer:
        model_down = depth_down.to(DEVICE)
        model = depth2normal.to(DEVICE)
    
    if target_task==tasks.normal:
        model_up = normal_up.to(DEVICE)
    if target_task==tasks.depth_zbuffer:
        model_up = depth_up.to(DEVICE)
    
    with torch.no_grad():
#         out = model(in_data)
        out = model_up(model_down(in_data))
    accuracy_tmp, _ = target_task.norm(out, out_data, compute_mse=False, batch_mean=False)
    accuracy += accuracy_tmp.sum()

print(accuracy / 8785. * 100.)

    rgb file len: 8785
    Intersection files len:  8785
tensor(5.5808, device='cuda:0')


## PERCEPTUAL

In [47]:
src_task = tasks.rgb
middle_task = tasks.depth_zbuffer
target_task = tasks.normal
tasks_set = [src_task, target_task]

buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

test_iter = iter(test_loader)

accuracy = 0
for in_data, out_data in test_iter:
    in_data = in_data.to(DEVICE)
    out_data = out_data.to(DEVICE)
    if src_task==tasks.rgb:
        model_down = rgb_down.to(DEVICE)
    if src_task==tasks.normal:
        model_down = normal_down.to(DEVICE)
    if src_task==tasks.depth_zbuffer:
        model_down = depth_down.to(DEVICE)
    
    if middle_task==tasks.normal:
        model_up = normal_up.to(DEVICE)
        model_down_per = normal_down.to(DEVICE)
        model_up_per = depth_up.to(DEVICE)
        model_per = normal2depth.to(DEVICE)
    if middle_task==tasks.depth_zbuffer:
        model_up = depth_up.to(DEVICE)
        model_down_per = depth_down.to(DEVICE)
        model_up_per = normal_up.to(DEVICE)
        model_per = depth2normal.to(DEVICE)
    
    with torch.no_grad():
        out = model_per(model_up(model_down(in_data)))
#         out = model_up_per(model_down_per(model_up(model_down(in_data))))
    accuracy_tmp, _ = target_task.norm(out, out_data, compute_mse=False, batch_mean=False)
    accuracy += accuracy_tmp.sum()

print(accuracy / 8785. * 100.)

    rgb file len: 8785
    Intersection files len:  8785
tensor(17.3254, device='cuda:0')


In [10]:
# buildings = ['almena']
# batch_size = 16
# test_loader = load_all([tasks.depth_zbuffer, tasks.normal], buildings, batch_size=batch_size)

# test_iter = iter(test_loader)
# in_data, out_data = next(test_iter)
# out = model_per(in_data)

    depth_zbuffer file len: 8785
    Intersection files len:  8785


In [11]:
shape = list(out_data.shape)
shape[1] = 3
out = out.clamp(min=0, max=1).expand(*shape)
out_data = out_data.clamp(min=0, max=1).expand(*shape)

In [12]:
images = [out_data, out]

logger.images_grouped(images, f"results:r, r(n(x))", resize=256)

# DATASET

In [6]:
tasks_set = [tasks.rgb, tasks.normal, tasks.depth_zbuffer]

In [6]:
# test_set = load_test(tasks_set, buildings=['almena', 'albertville', 'espanola'])
# test = RealityTask.from_static(
#     "test", test_set, tasks_set
# )

In [7]:
buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

    rgb file len: 8785
    Intersection files len:  8785


In [8]:
#test_iter = iter(test_loader)
#rgb_data, normal_data, depth_data = next(test_iter)

# VISUALIZE RESULTS

In [68]:
rgb_data = test.task_data[tasks.rgb].to(DEVICE)
normal_data = test.task_data[tasks.normal].to(DEVICE)
depth_data = test.task_data[tasks.depth_zbuffer].to(DEVICE)

In [69]:
with torch.no_grad():
    rgb_out = rgb_down(rgb_data)
    normal_out = normal_up(rgb_out)
    depth_out = depth_up(rgb_out)
    
    shape = list(rgb_data.shape)
    shape[1] = 3
    normal_out = normal_out.clamp(min=0, max=1).expand(*shape)
    depth_out = depth_out.clamp(min=0, max=1).expand(*shape)
    depth_data = depth_data.clamp(min=0, max=1).expand(*shape)

In [70]:
images = [depth_data, depth_out, normal_data, normal_out]

logger.images_grouped(images, f"results:r, r(x), n, n(x)", resize=256)